In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Library

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00


In [3]:
!pip install torch

# Import Library

In [4]:
from datasets import load_dataset
from torch.utils.data import random_split, DataLoader, Dataset
from collections import Counter
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from transformers import pipeline

# Load Dataset

In [5]:
dataset = load_dataset("fhirfly/medicalquestions", 'train')
dataset = dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
print(dataset[:10])
print(type(dataset))

{'label': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'text': ['What is (are) Anal Cancer ?', 'How to prevent Anal Cancer ?', 'What is (are) Adult Central Nervous System Tumors ?', 'What is (are) Childhood Brain and Spinal Cord Tumors ?', 'What is (are) Childhood Astrocytomas ?', 'What is (are) Childhood Central Nervous System Atypical Teratoid/Rhabdoid Tumor ?', 'What is (are) Childhood Brain Stem Glioma ?', 'What is (are) Childhood Central Nervous System Embryonal Tumors ?', 'What is (are) Childhood Central Nervous System Germ Cell Tumors ?', 'What is (are) Childhood Craniopharyngioma ?']}
<class 'datasets.arrow_dataset.Dataset'>


# Train Test Split Dataset

In [7]:
# Tentukan ukuran untuk set pelatihan dan validasi
training_size = int(0.8 * len(dataset))
testing_size = len(dataset) - training_size

# Bagi dataset menjadi set pelatihan dan validasi
training_dataset, testing_dataset = random_split(dataset, [training_size, testing_size])

print(f"Jumlah data pelatihan: {len(training_dataset)}")
print(f"Jumlah data validasi: {len(testing_dataset)}")


Jumlah data pelatihan: 20008
Jumlah data validasi: 5003


In [8]:
print(training_dataset[:10])
print(type(training_dataset))

{'label': [1, 1, 0, 1, 1, 1, 1, 0, 0, 1], 'text': ['What are the symptoms of Spinal muscular atrophy type 3 ?', 'Do you have information about Drug Safety', 'When was the Mona Lisa painted?', 'What is (are) Andermann syndrome ?', 'When should I (you) have a (an) Colostomy or skin level cecostomy ?', 'What are the symptoms of Hereditary hemorrhagic telangiectasia type 4 ?', 'Why do I have compulsions?', 'what weight division is moe herscovitch in', 'what kind of game is p.t.o.', 'What is (are) the active ingredient in the drug lysine?']}
<class 'torch.utils.data.dataset.Subset'>


In [9]:
listTrainingDataset = list(training_dataset)
labelDatatraining = [training_dataset[i]['label'] for i in range(len(training_dataset))]
textDatatraining = [training_dataset[i]['text'] for i in range(len(training_dataset))]
listTestingDataset = list(testing_dataset)
labelDatatesting = [testing_dataset[i]['label'] for i in range(len(testing_dataset))]
textDatatesting = [testing_dataset[i]['text'] for i in range(len(testing_dataset))]

In [10]:
uniqueDataTraining = set(labelDatatraining)
print(uniqueDataTraining)
uniqueTotalTraining = Counter(labelDatatraining)
print(uniqueTotalTraining)

uniqueDataTesting = set(labelDatatesting)
print(uniqueDataTesting)
uniqueTotalTesting = Counter(labelDatatesting)
print(uniqueTotalTesting)

{0, 1}
Counter({0: 10201, 1: 9807})
{0, 1}
Counter({0: 2553, 1: 2450})


In [11]:
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []

In [12]:
for s in training_dataset:
    training_sentences.append(s.get('text'))
for l in training_dataset:
    training_labels.append(l.get('label'))
for s in testing_dataset:
    testing_sentences.append(s.get('text'))
for l in testing_dataset:
    testing_labels.append(l.get('label'))

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

In [13]:
print(type(training_labels[:10]))

<class 'list'>


In [14]:
pipe = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [15]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# Create and fit the tokenizer with training data
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# Tokenisasi data
train_encodings = tokenizer(training_sentences, truncation=True, padding=True)
val_encodings = tokenizer(testing_sentences, truncation=True, padding=True)

# Buat dataset dan dataloader
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = CustomDataset(train_encodings, training_labels)
val_dataset = CustomDataset(val_encodings, testing_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Pilih perangkat (CPU atau GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Inisialisasi optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Fine-tuning loop
num_epochs = 5  # Ganti dengan jumlah epoch yang sesuai
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Evaluasi pada set validasi (opsional)
    model.eval()
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        val_loss = outputs.loss

    print(f"Epoch {epoch + 1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

# Simpan model yang telah di-fine-tune (opsional)
model.save_pretrained("path/to/your/fine_tuned_model")
tokenizer.save_pretrained("path/to/your/fine_tuned_model")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Training Loss: 0.0003489467198960483, Validation Loss: 0.0001956373598659411
Epoch 2, Training Loss: 0.00015097437426447868, Validation Loss: 7.962757808854803e-05
Epoch 3, Training Loss: 9.008684719447047e-05, Validation Loss: 4.0808383346302435e-05
Epoch 4, Training Loss: 3.5746823414228857e-05, Validation Loss: 1.2715576303889975e-05
Epoch 5, Training Loss: 1.3366241546464153e-05, Validation Loss: 8.980392522062175e-06


('path/to/your/fine_tuned_model/tokenizer_config.json',
 'path/to/your/fine_tuned_model/special_tokens_map.json',
 'path/to/your/fine_tuned_model/vocab.txt',
 'path/to/your/fine_tuned_model/added_tokens.json',
 'path/to/your/fine_tuned_model/tokenizer.json')

In [16]:
# Lakukan prediksi
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Hitung loss
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(logits, labels)

# Hitung akurasi
predictions = torch.argmax(logits, dim=1)
correct_predictions = torch.sum(predictions == labels).item()
total_samples = len(labels)
accuracy = correct_predictions / total_samples * 100

# Print hasil
print(f"Loss: {loss.item()}")
print(f"Accuracy: {accuracy:.2f}%")


Loss: 8.980392522062175e-06
Accuracy: 100.00%


In [17]:
# Ganti path dengan path model yang benar
model_path = "path/to/your/fine_tuned_model"

# Muat model dan tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [18]:
new_data = ["how was your day?"]

# Gunakan encode_plus untuk mendapatkan input yang sesuai dengan model
encoding = tokenizer.encode_plus(
    new_data[0],
    truncation=True,
    padding=True,
    return_tensors="pt"  # Mengembalikan PyTorch tensors
)

input_ids = encoding["input_ids"]
attention_mask = encoding["attention_mask"]

# Pindahkan ke perangkat yang sesuai (CPU atau GPU)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

In [19]:
# Pindahkan input ke perangkat yang sesuai (CPU atau GPU)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Lakukan prediksi
with torch.no_grad():
    # Pastikan model juga berada di perangkat yang sesuai
    model = model.to(device)

    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Ambil prediksi kelas
predicted_class = torch.argmax(logits, dim=1).item()

print(f"Predicted Class: {predicted_class}")


Predicted Class: 0
